In [1]:
# clone and install the required packages for the Face Recognition project
!git clone https://github.com/sathishkumar67/Face-Recognition-using-Resnet.git
!mv /kaggle/working/Face-Recognition-using-Resnet/* /kaggle/working/
!pip install --upgrade pip
!pip install -r requirements.txt
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Cloning into 'Face-Recognition-using-Resnet'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 117 (delta 57), reused 82 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 35.55 MiB | 3.02 MiB/s, done.
Resolving deltas: 100% (57/57), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 52.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.8 MB/s eta 0:00:00
  Atte

In [2]:
# Import necessary libraries
import os
import random
import numpy as np
import torch
from huggingface_hub import hf_hub_download
from siamese_resnet.utils import unzip_file
from siamese_resnet.dataset import TripletDataset, TripletDatasetGenerator, triplet_collate_fn
from siamese_resnet.model import SiameseResNet
from siamese_resnet.loss import TripletLoss
from siamese_resnet.trainer import train_model

# Define constants
DATASET_REPO_ID = "pt-sk/Face_Recognition_Dataset"
DATA_ROOT = "face_recognition_with_cropped_faces_dataset"
DATASET_FILENAME_IN_REPO = f"{DATA_ROOT}.zip"
DATASET_REPO_TYPE = "dataset"
LOCAL_DIR = os.getcwd()
NUM_CORES = os.cpu_count()
SEED = 42
DATALOADER_PIN_MEMORY_DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set the random seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Download the dataset from Hugging Face Hub
hf_hub_download(repo_id=DATASET_REPO_ID, filename=DATASET_FILENAME_IN_REPO, repo_type=DATASET_REPO_TYPE, local_dir=LOCAL_DIR)

# Unzip the dataset
unzip_file(os.path.join(LOCAL_DIR, DATASET_FILENAME_IN_REPO), LOCAL_DIR)

2025-05-13 02:18:22.676043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747102702.699413      72 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747102702.706308      72 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747102702.724996      72 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747102702.725017      72 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747102702.725019      72 computation_placer.cc:177] computation placer alr

(…)cognition_with_cropped_faces_dataset.zip:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 1.31G/1.31G [00:14<00:00, 91.4MB/s]


Unzipped /kaggle/working/face_recognition_with_cropped_faces_dataset.zip to /kaggle/working
Removed zip file: /kaggle/working/face_recognition_with_cropped_faces_dataset.zip


In [3]:
# Initialize generator
generator = TripletDatasetGenerator(DATA_ROOT)

# Create splits
splits = generator.create_splits()
    
# Generate triplets for each split
train_triplets = generator.generate_triplets(splits['train'])
val_triplets = generator.generate_triplets(splits['val'])
test_triplets = generator.generate_triplets(splits['test'])

# Create datasets
train_dataset = TripletDataset(train_triplets)
val_dataset = TripletDataset(val_triplets)
test_dataset = TripletDataset(test_triplets)

# Create dataloaders (4 workers for optimal IO)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, 
                        num_workers=NUM_CORES, pin_memory=True, persistent_workers=True,
                        collate_fn=triplet_collate_fn, prefetch_factor=2, pin_memory_device=DATALOADER_PIN_MEMORY_DEVICE)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False,
                        num_workers=NUM_CORES, pin_memory=True, persistent_workers=True,
                        collate_fn=triplet_collate_fn, prefetch_factor=2, pin_memory_device=DATALOADER_PIN_MEMORY_DEVICE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False,
                        num_workers=NUM_CORES, pin_memory=True, persistent_workers=True,
                        collate_fn=triplet_collate_fn, prefetch_factor=2, pin_memory_device=DATALOADER_PIN_MEMORY_DEVICE)

In [4]:
model = SiameseResNet(embedding_dim=256)
# model.to(DEVICE)
# criterion = TripletLoss(margin=0.5)
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=3e-4, betas=(0.9, 0.999))

# history, model = train_model(
#     model=model,
#     device=DEVICE,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     criterion=criterion,
#     optimizer=optimizer,
#     epochs=100,
#     patience=10,
#     checkpoint_path='best_face_model.pth'
# )   

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 75.4MB/s]


In [10]:
import torch.nn as nn
model.backbone.fc = nn.Identity()

In [12]:
model(torch.randn(1, 3, 224, 224)).shape

torch.Size([1, 512])

In [13]:
model

SiameseResNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr